<a href="https://colab.research.google.com/github/MorningStarTM/IMDB-movie-review-sentiment-analysis-/blob/main/imdb_movie_review_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.layers import ReLU
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
#read data
data = pd.read_csv("/content/drive/MyDrive/DataSet/csv dataset/IMDB Dataset.csv")

In [6]:
#size of data
data.shape

(50000, 2)

In [7]:
data.columns

Index(['review', 'sentiment'], dtype='object')

In [8]:
#check null value 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [9]:
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [10]:
df = data[0:5000]

In [11]:
df.shape

(5000, 2)

# Vectorization

In [15]:
#vocabular size
voc_size = 5000

In [16]:
sent_length = 200

In [17]:
df['review'] = df['review'].apply(lambda x: x.lower())
df['review'] = df['review'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['review'].values)
X = tokenizer.texts_to_sequences(df['review'].values)
X = pad_sequences(X)

<ipython-input-17-6c908cd7c2d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: x.lower())
<ipython-input-17-6c908cd7c2d9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
/usr/local/lib/python3.9/dist-packages/keras/preprocessing/text.py:246: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn(


In [18]:
Y = pd.get_dummies(df['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print('Shape of training samples:',X_train.shape,Y_train.shape)
print('Shape of testing samples:',X_test.shape,Y_test.shape)

Shape of training samples: (3350, 1319) (3350, 2)
Shape of testing samples: (1650, 1319) (1650, 2)


In [20]:
Y_train[0:10]

array([[1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1]], dtype=uint8)

# Model

In [21]:
X.shape[1]

1319

In [34]:
model = Sequential()
model.add(Embedding(max_fatures, 128 ,input_length = X.shape[1]))
model.add(LSTM(128))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1319, 128)         256000    
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dense_3 (Dense)             (None, 2)                 258       
                                                                 
Total params: 387,842
Trainable params: 387,842
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=5, batch_size=16)

Epoch 1/5
210/210 [==============================] - 21s 102ms/step - loss: 0.4503 - accuracy: 0.7982 - val_loss: 0.4493 - val_accuracy: 0.8000
Epoch 2/5
210/210 [==============================] - 19s 91ms/step - loss: 0.3570 - accuracy: 0.8567 - val_loss: 0.4558 - val_accuracy: 0.8000
Epoch 3/5
210/210 [==============================] - 15s 73ms/step - loss: 0.3193 - accuracy: 0.8615 - val_loss: 0.5984 - val_accuracy: 0.6958
Epoch 4/5
210/210 [==============================] - 14s 66ms/step - loss: 0.2441 - accuracy: 0.9039 - val_loss: 0.5790 - val_accuracy: 0.7685
Epoch 5/5
210/210 [==============================] - 13s 61ms/step - loss: 0.1601 - accuracy: 0.9418 - val_loss: 0.5986 - val_accuracy: 0.7952


# Evaluating Model

In [37]:
model.evaluate(X_test, Y_test)

52/52 [==============================] - 1s 20ms/step - loss: 0.5986 - accuracy: 0.7952


[0.5986361503601074, 0.7951515316963196]

In [64]:
prediction = model.predict(X_test)

52/52 [==============================] - 2s 20ms/step


In [79]:
result = []
for i in range(len(prediction)):
  t = np.argmax(prediction[0])
  result.append(t)

In [82]:
temp_1 = []
for i in range(1650):
  if Y_test[i][0] == 1:
    temp_1.append(1)
  else:
    temp_1.append(0)

In [84]:
cm = confusion_matrix(temp_1, result)

In [85]:
cm

array([[791,   0],
       [859,   0]])